# MovieLens Preprocessing: Build cleaned_movies.csv

This notebook loads the MovieLens small dataset, cleans movie titles and genres, computes per-movie average ratings and normalized scores, joins TMDb IDs for posters, and saves a compact `cleaned_movies.csv` for the Streamlit hybrid recommender.

Datasets used (from `ml-latest-small 2/`):
- `movies.csv` (movieId, title, genres)
- `ratings.csv` (userId, movieId, rating, timestamp)
- `links.csv` (movieId, imdbId, tmdbId)

Output:
- `/workspace/cleaned_movies.csv` with columns: `movieId`, `tmdbId`, `title` (lowercase), `genres_text`, `avg_rating`, `norm_rating`, `rating_count`.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path('ml-latest-small 2')
MOVIES_CSV = DATA_DIR / 'movies.csv'
RATINGS_CSV = DATA_DIR / 'ratings.csv'
LINKS_CSV = DATA_DIR / 'links.csv'
OUTPUT_CSV = Path('cleaned_movies.csv')


In [ ]:
movies = pd.read_csv(MOVIES_CSV)
ratings = pd.read_csv(RATINGS_CSV)
links = pd.read_csv(LINKS_CSV)
movies.head(), ratings.head(), links.head()


In [ ]:
# Clean titles and genres
movies['title'] = movies['title'].fillna('').str.strip().str.lower()
movies['genres'] = movies['genres'].fillna('(no genres listed)')
# Split and normalize genres
movies['genres_list'] = movies['genres'].str.split('|')
movies['genres_list'] = movies['genres_list'].apply(lambda lst: [g.strip().lower().replace('-', ' ') for g in lst] if isinstance(lst, list) else [])
movies['genres_text'] = movies['genres_list'].apply(lambda lst: ' '.join(sorted(set(lst))))
movies[['movieId','title','genres_text']].head()


In [ ]:
# Compute average rating and counts per movie
ratings = ratings.dropna(subset=['movieId','rating'])
agg = ratings.groupby('movieId').agg(avg_rating=('rating','mean'), rating_count=('rating','size')).reset_index()
# Normalize avg_rating between 0 and 1
if not agg.empty:
    min_r, max_r = agg['avg_rating'].min(), agg['avg_rating'].max()
    if max_r > min_r:
        agg['norm_rating'] = (agg['avg_rating'] - min_r) / (max_r - min_r)
    else:
        agg['norm_rating'] = 0.5
else:
    agg['norm_rating'] = pd.Series(dtype=float)
agg.head()


In [ ]:
# Merge datasets
df = movies.merge(links[['movieId','tmdbId']], on='movieId', how='left')
df = df.merge(agg, on='movieId', how='left')
# Fill missing ratings
df['avg_rating'] = df['avg_rating'].fillna(0.0)
df['norm_rating'] = df['norm_rating'].fillna(0.0)
df['rating_count'] = df['rating_count'].fillna(0).astype(int)
# Clean tmdbId to int where possible
def to_int_or_none(x):
    try:
        xi = int(x)
        return xi if xi > 0 else np.nan
    except Exception:
        return np.nan
df['tmdbId'] = df['tmdbId'].apply(to_int_or_none)
# Select columns and save
out_cols = ['movieId','tmdbId','title','genres_text','avg_rating','norm_rating','rating_count']
cleaned = df[out_cols].copy()
cleaned.to_csv(OUTPUT_CSV, index=False)
cleaned.head(10)


Now you can run the Streamlit app which loads `cleaned_movies.csv`.

In [ ]:
# Visualization: Top Genres (bar) and Rating Distribution (hist)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

# Ensure `cleaned` exists; if not, read from disk
try:
    cleaned
except NameError:
    cleaned = pd.read_csv('cleaned_movies.csv')

# Top genres bar plot
genres_flat = []
for g in cleaned['genres_text'].fillna(''):
    genres_flat += str(g).split()

if genres_flat:
    genre_counts = pd.Series(genres_flat).value_counts().head(10)
    fig, ax = plt.subplots(figsize=(8, 4))
    sns.barplot(x=genre_counts.values, y=genre_counts.index, ax=ax, palette="Blues_r")
    ax.set_title('Top 10 Genres')
    ax.set_xlabel('Count')
    ax.set_ylabel('Genre')
    plt.show()

# Rating distribution histogram (average rating per movie)
fig2, ax2 = plt.subplots(figsize=(7, 4))
sns.histplot(cleaned['avg_rating'], bins=20, kde=False, ax=ax2, color="#4c72b0")
ax2.set_title('Average Rating Distribution (per Movie)')
ax2.set_xlabel('Average Rating')
ax2.set_ylabel('Number of Movies')
plt.show()

In [ ]:
# Pairplot of rating metrics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")

try:
    cleaned
except NameError:
    cleaned = pd.read_csv('cleaned_movies.csv')

pair_cols = ['avg_rating', 'norm_rating', 'rating_count']
subset = cleaned[pair_cols].copy()
# Avoid extremely skewed scales by clipping rating_count for nicer plots
subset['rating_count_clipped'] = subset['rating_count'].clip(upper=subset['rating_count'].quantile(0.99))

sns.pairplot(
    subset.rename(columns={'rating_count_clipped': 'rating_count (clipped)'}),
    vars=['avg_rating', 'norm_rating', 'rating_count (clipped)'],
    diag_kind='hist',
    corner=True
)
plt.suptitle('Pairplot: avg_rating, norm_rating, rating_count', y=1.02)
plt.show()

In [ ]:
# Train/test split for ratings and save to disk
from sklearn.model_selection import train_test_split
from pathlib import Path

# Use cleaned ratings (dropna already applied earlier)
try:
    ratings
except NameError:
    import pandas as pd
    ratings = pd.read_csv('ml-latest-small 2/ratings.csv')
    ratings = ratings.dropna(subset=['movieId','rating'])

TRAIN_CSV = Path('ml-latest-small 2') / 'ratings_train.csv'
TEST_CSV = Path('ml-latest-small 2') / 'ratings_test.csv'

ratings_train, ratings_test = train_test_split(ratings, test_size=0.2, random_state=42)

ratings_train.to_csv(TRAIN_CSV, index=False)
ratings_test.to_csv(TEST_CSV, index=False)

len(ratings_train), len(ratings_test), TRAIN_CSV, TEST_CSV

In [ ]:
# Recommender utilities for notebook use
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load cleaned data if not available
try:
    cleaned
except NameError:
    cleaned = pd.read_csv('cleaned_movies.csv')

# Build TF-IDF on genres_text (content-based)
_tf_vec = TfidfVectorizer(stop_words='english')
_tfidf = _tf_vec.fit_transform(cleaned['genres_text'].fillna(''))

# Build user–item pivot from original ratings
ratings_path = 'ml-latest-small 2/ratings.csv'
ratings_nb = pd.read_csv(ratings_path, usecols=['userId','movieId','rating'])
pivot_nb = ratings_nb.pivot_table(index='userId', columns='movieId', values='rating', aggfunc='mean').fillna(0.0)

# Ensure pivot columns align to cleaned movieId order
movie_ids = cleaned['movieId']
missing_cols = [m for m in movie_ids if m not in pivot_nb.columns]
for m in missing_cols:
    pivot_nb[m] = 0.0
pivot_nb = pivot_nb.loc[:, movie_ids]

from numpy.linalg import norm

def recommend_content(title: str, k: int = 10) -> pd.DataFrame:
    idx_list = cleaned.index[cleaned['title'] == title.lower()].tolist()
    if not idx_list:
        raise ValueError('Title not found')
    idx = idx_list[0]
    sims = cosine_similarity(_tfidf[idx], _tfidf).ravel()
    df = cleaned.copy()
    df['score'] = sims
    df = df[df.index != idx]
    return df.sort_values('score', ascending=False).head(k)[['title','genres_text','avg_rating','score']]


def recommend_item_item(title: str, k: int = 10) -> pd.DataFrame:
    idx_list = cleaned.index[cleaned['title'] == title.lower()].tolist()
    if not idx_list:
        raise ValueError('Title not found')
    idx = idx_list[0]
    target_vec = pivot_nb.iloc[:, idx].values.astype(float)
    M = pivot_nb.values.astype(float)
    num = M.T @ target_vec
    den = (norm(M, axis=0) * norm(target_vec) + 1e-12)
    sims = np.nan_to_num(num / den)
    df = cleaned.copy()
    df['score'] = sims
    df = df[df.index != idx]
    return df.sort_values('score', ascending=False).head(k)[['title','genres_text','avg_rating','score']]


def recommend_hybrid(title: str, k: int = 10, w_content: float = 0.7, w_cf: float = 0.3) -> pd.DataFrame:
    idx_list = cleaned.index[cleaned['title'] == title.lower()].tolist()
    if not idx_list:
        raise ValueError('Title not found')
    idx = idx_list[0]
    content = cosine_similarity(_tfidf[idx], _tfidf).ravel()
    target_vec = pivot_nb.iloc[:, idx].values.astype(float)
    M = pivot_nb.values.astype(float)
    num = M.T @ target_vec
    den = (norm(M, axis=0) * norm(target_vec) + 1e-12)
    item = np.nan_to_num(num / den)
    final = w_content * content + w_cf * item
    df = cleaned.copy()
    df['score'] = final
    df = df[df.index != idx]
    return df.sort_values('score', ascending=False).head(k)[['title','genres_text','avg_rating','score']]

In [ ]:
# Build item–item CF pivot on train split
import pandas as pd
import numpy as np
from numpy.linalg import norm

ratings_train = pd.read_csv('ml-latest-small 2/ratings_train.csv')

# Align to cleaned movieIds
pivot_train = ratings_train.pivot_table(index='userId', columns='movieId', values='rating', aggfunc='mean').fillna(0.0)
movie_ids = cleaned['movieId']
missing_cols = [m for m in movie_ids if m not in pivot_train.columns]
for m in missing_cols:
    pivot_train[m] = 0.0
pivot_train = pivot_train.loc[:, movie_ids]

# Helper for item similarity from train
M_train = pivot_train.values.astype(float)

def item_sim_vector_from_train(item_index: int) -> np.ndarray:
    target_vec = M_train[:, item_index]
    num = M_train.T @ target_vec
    den = (norm(M_train, axis=0) * norm(target_vec) + 1e-12)
    return np.nan_to_num(num / den)

print('Train pivot shape (users x items):', pivot_train.shape)

In [ ]:
# Demo: Try content-based, item-item CF, and hybrid
# Choose a title (lowercase) that exists in cleaned['title']
example_title = 'toy story (1995)'.lower()

try:
    print('Content-based:')
    display(recommend_content(example_title, k=10))
    print('\nItem-item CF:')
    display(recommend_item_item(example_title, k=10))
    print('\nHybrid (0.7 content, 0.3 CF):')
    display(recommend_hybrid(example_title, k=10, w_content=0.7, w_cf=0.3))
except ValueError as e:
    print('Error:', e)
    print('Available example titles:', cleaned['title'].head(10).tolist())